In [ ]:
%%shell
pip install cython
pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
mkdir dataset
%%shell
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0
cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-b2nkwco6
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-b2nkwco6
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=266453 sha256=68ab3277d7dca822748663c3dc73bd098a493299e27bd074cd5626babad10f78
  Stored in directory: /tmp/pip-ephem-wheel-cache-fg5w4z4b/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0
    Uninstalling pycocotools-2.0:
      Successfully uninstalled pycocotools-2.0
/bin/bash: line 3: fg: no job control
fatal: destination path 'vision' already exists and is not an empty directory.
HEAD is now at be37608 version check against PyTorch's CUDA version


In [ ]:
import os
import csv
import numpy as np
import torch
import torch.utils.data
from PIL import Image
import xml.etree.ElementTree as ET
from engine import train_one_epoch, evaluate
import utils
import transforms as T
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [ ]:
def remove_bad_photos(root_dir):
  annotations = list(sorted(os.listdir(os.path.join(root_dir, "Annotations")))) 
  images = list(sorted(os.listdir(os.path.join(root_dir, "Images"))))
  for ann in annotations:
    ann_path = os.path.join(root_dir, "Annotations", ann)
    img_path = os.path.join(root_dir, "Images", os.path.splitext(os.path.basename(ann_path))[0] +'.jpg')
    LList = []
    LL = []
    line_count = 0
    with open(ann_path,'r') as csv_file:
	    csv_reader = csv.reader(csv_file, delimiter=',')
	    for row in csv_reader:
		    LList = []
		    for i in row:
			    data = i.split(" ")
			    for j in data:
				    LList.append(int(j))
		    LL.append(LList)
		    line_count += 1
    boxes = np.array(LL[1:])
    num_objs = LL[0][0]
    if num_objs < 1:
      os.remove(ann_path)
      os.remove(img_path)

In [ ]:
remove_bad_photos('dataset/')

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "Images"))))
        self.annotations = list(sorted(os.listdir(os.path.join(root, "Annotations"))))

    def __getitem__(self, idx):
        # load images ad masks
        img_path = os.path.join(self.root, "Images", self.imgs[idx])
        ann_path = os.path.join(self.root, "Annotations", self.annotations[idx])
        img = Image.open(img_path).convert("RGB")

        LList = []
        LL = []
        line_count = 0
        with open(ann_path,'r') as csv_file:
	        csv_reader = csv.reader(csv_file, delimiter=',')
	        for row in csv_reader:
		        LList = []
		        for i in row:
			        data = i.split(" ")
			        for j in data:
				        LList.append(int(j))
		        LL.append(LList)
		        line_count += 1
        boxes = np.array(LL[1:])
        num_objs = LL[0][0]
        labels = torch.ones((num_objs,), dtype=torch.int64)
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        image_id = torch.tensor([idx])
        
        if num_objs < 1:
          target = {}
          target["boxes"] = torch.as_tensor(np.zeros(4), dtype=torch.float32)
          target["labels"] = labels
          target["image_id"] = image_id
          target["area"] = 0
          target["iscrowd"] = iscrowd

          if self.transforms is not None:
              img, target = self.transforms(img, target)
          return img, target

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [ ]:
class CustomDatasetTest(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(os.path.join(root, "Images"))))

    def __getitem__(self, idx):
        # load images ad masks
        img_path = os.path.join(self.root, "Images", self.imgs[idx])
        img = Image.open(img_path).convert("RGB")
        target = {}
        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [ ]:
def Model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features,  num_classes)
    return model

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
dataset = CustomDataset('dataset/', get_transform(train=True))
dataset_test = CustomDataset('dataset/', get_transform(train=False))
num_classes = 20

torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[0:-50])
indices = torch.randperm(len(dataset)).tolist()
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

data_loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True, num_workers=4, collate_fn=utils.collate_fn)
data_loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=1, shuffle=False, num_workers=4, collate_fn=utils.collate_fn)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = Model(num_classes)
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

num_epochs = 10
for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    lr_scheduler.step()
    evaluate(model, data_loader_test, device=device)

In [ ]:
img,_ = dataset_test[0]
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])

img = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
box = prediction[0]['boxes'].cpu().numpy()
scores = prediction[0]['scores'].cpu().numpy()
fig,ax = plt.subplots(1)
ax.imshow(img)
rect = []
for i in range (int(box.size/4)):
  if scores[i]>0.3:
    rect = patches.Rectangle((box[i][0], box[i][1]),(box[i][2]-box[i][0]),(box[i][3]-box[i][1]),linewidth=1,edgecolor='r',facecolor='none')
    ax.add_patch(rect)

plt.show()